In [ ]:
# import os
# from datetime import datetime
# from dotenv import load_dotenv
# #
# # #--------Google Drive Integration--------#
# # # from google.colab import drive, userdata
# # # This gives Colab access to your files in Google Drive.
# # # drive.mount('/content/drive')
# # # 'GITHUB_USERNAME' and 'GITHUB_TOKEN' saved as secrets in Colab.
# # GITHUB_USERNAME = userdata.get('GITHUB_USERNAME')
# # GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
# # REPOSITORY_NAME = 'PyNucleus-Model' # Your repository name
# # NOTEBOOK_DRIVE_PATH = "/content/drive/MyDrive/PyNucleus Project/Capstone Project.ipynb"
# #
# #
# # #--------Cursor Integration--------#
# # # Load environment variables from .env file
# load_dotenv()
# #
# # # Get GitHub credentials from environment variables
# GITHUB_USERNAME = os.getenv('GITHUB_USERNAME')
# GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
# #
# # # Print to verify the variables are loaded (remove this in production)
# print(f"Username: {GITHUB_USERNAME}")
# print(f"Token: {GITHUB_TOKEN[:4]}...") # Only print first 4 chars of token for security
# #
# # Repository information
# REPOSITORY_NAME = 'PyNucleus-Model'
# NOTEBOOK_REPO_FILENAME = "Capstone Project.ipynb"
# LOG_FILENAME = "update_log.txt"

# # Pull latest changes from GitHub
# print("Pulling latest changes from GitHub...")
# !git pull https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git main

# print("Repository is up to date!")

# # Log start time
# with open("update_log.txt", "a") as f:
#     f.write(f" {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Log Update\n")

In [47]:
# PyNucleus Model - Setup and Imports
import sys
import os
from pathlib import Path

# Add project root to path
sys.path.append(os.path.abspath('.'))

# Import PyNucleus Pipeline modules
from core_modules.pipeline import RAGPipeline, DWSIMPipeline, ResultsExporter, PipelineUtils

print(" PyNucleus Model - Pipeline Ready!")
print(" Available Components:")
print("   • RAGPipeline - Document processing and retrieval")
print("   • DWSIMPipeline - Chemical process simulation") 
print("   • ResultsExporter - CSV export functionality")
print("   • PipelineUtils - Complete pipeline orchestration")

 PyNucleus Model - Pipeline Ready!
 Available Components:
   • RAGPipeline - Document processing and retrieval
   • DWSIMPipeline - Chemical process simulation
   • ResultsExporter - CSV export functionality
   • PipelineUtils - Complete pipeline orchestration


# **PyNucleus Model - Complete Pipeline**

This notebook contains the complete PyNucleus model pipeline with separate sections for:
1. **Data Ingestion and Preprocessing for RAG** 
2. **DWSIM Integration and Simulation**
3. **Results Export to CSV**

In [ ]:
# Initialize Pipeline Components
pipeline = PipelineUtils()

print("\n🔧 Pipeline Initialized!")
print("Available Functions:")
print("   • pipeline.run_complete_pipeline() - Run everything")
print("   • pipeline.run_rag_only() - RAG pipeline only")  
print("   • pipeline.run_dwsim_only() - DWSIM simulations only")
print("   • pipeline.quick_test() - Verify status")
print("   • pipeline.view_results_summary() - View results")
print("   • pipeline.print_pipeline_status() - Detailed status")
print("   • pipeline.clean_all_results() - Clean all data")


In [48]:
# ========================================
# SECTION 1: COMPLETE PIPELINE - Run Everything
# ========================================

# Run the complete pipeline (RAG + DWSIM + Export)
results = pipeline.run_complete_pipeline()

# Display results summary
if results:
    print(f"\n🎉 Pipeline completed in {results['duration']:.1f} seconds!")
    print(f"📊 RAG Results: {len(results['rag_data'])} queries processed")
    print(f"🔬 DWSIM Results: {len(results['dwsim_data'])} simulations completed")
    print(f"📁 Exported Files: {len(results['exported_files'])} CSV files created")
else:
    print("❌ Pipeline execution failed")

🚀 Running complete PyNucleus pipeline...
🗑️ RAG results cleared.
🗑️ DWSIM results cleared.
📚 Starting RAG Pipeline...
Step 1: Processing source documents...
--- 📄 Starting processing for 5 file(s) in '/Users/mohammadalmusaiteer/PyNucleus-Model/source_documents' ---


Processing files:   0%|          | 0/5 [00:00<?, ?it/s]WARNING: libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


 ▶ Processing: Manuscript Draft_Can Modular Plants Lower African Industrialization Barriers.docx
   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/converted_to_txt/Manuscript Draft_Can Modular Plants Lower African Industrialization Barriers.txt
 ▶ Processing: mcp_basics.txt
   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/converted_to_txt/mcp_basics.txt
 ▶ Processing: feasibility_factors.txt
   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/converted_to_txt/feasibility_factors.txt
 ▶ Processing: Bist_Madan.pdf


Processing files: 100%|██████████| 5/5 [00:00<00:00, 10.90it/s]

   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/converted_to_txt/Bist_Madan.txt
 ▶ Processing: sample_document.txt
   • Success! Saved to: /Users/mohammadalmusaiteer/PyNucleus-Model/converted_to_txt/sample_document.txt

 All files processed.

Step 2: Scraping Wikipedia articles...
🔍 Starting Wikipedia article search for 5 keywords...
▶️  Searching for: modular design


✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/web_sources/wikipedia_modular_design.txt
▶️  Searching for: software architecture
✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/web_sources/wikipedia_software_architecture.txt
▶️  Searching for: system design
✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/web_sources/wikipedia_system_design.txt
▶️  Searching for: industrial design
✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/web_sources/wikipedia_industrial_design.txt
▶️  Searching for: supply chain


INFO: Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/web_sources/wikipedia_supply_chain.txt

✨ Article scraping complete!

Step 3: Processing and chunking documents...
📰 Found 5 Wikipedia articles
📄 Found 5 converted documents
📋 Total documents loaded: 10
✂️ Split into 846 chunks

✅ Successfully saved chunked data to /Users/mohammadalmusaiteer/PyNucleus-Model/converted_chunked_data/:
  • chunked_data_full.json - Complete data with metadata
  • chunked_data_stats.json - Statistical analysis
  • chunked_data_content.txt - Human-readable content


Step 4: Building FAISS vector store...
=== FAISS VectorDB Analysis ===
Started: 2025-06-10 16:07:26
Loaded 846 documents from /Users/mohammadalmusaiteer/PyNucleus-Model/converted_chunked_data/chunked_data_full.json
Embedding device → cpu   | dim=384
Docs indexed : 846
Index file   : /Users/mohammadalmusaiteer/PyNucleus-Model/chunk_reports/pynucleus_mcp.faiss
Embeds .pkl  : /Users/mohammadalmusaiteer/PyNucleus-Model/chunk_reports/embed

In [ ]:
# ========================================
# SECTION 2: INDIVIDUAL PIPELINE COMPONENTS
# ========================================

# Option 1: Run only RAG Pipeline
print("📚 RAG Only Pipeline:")
rag_results = pipeline.run_rag_only()
if rag_results:
    print(f"   ✅ {len(rag_results['rag_data'])} RAG queries processed")

print("\n" + "="*50 + "\n")

# Option 2: Run only DWSIM Simulations  
print("🔬 DWSIM Only Pipeline:")
dwsim_results = pipeline.run_dwsim_only()
if dwsim_results:
    print(f"   ✅ {len(dwsim_results['dwsim_data'])} DWSIM simulations completed")


In [ ]:
# ========================================
# SECTION 3: UTILITY FUNCTIONS
# ========================================

# View pipeline status
pipeline.print_pipeline_status()

print("\n" + "="*50 + "\n")

# View results summary
pipeline.view_results_summary()

print("\n" + "="*50 + "\n")

# Quick test
test_results = pipeline.quick_test()
print(f"✅ Quick test completed! Found {test_results['csv_files_count']} CSV files")


In [ ]:
# ========================================
# SECTION 4: SIMPLE FUNCTION CALLS
# ========================================

print("🎯 Simple Function Calls for Easy Testing:")
print()

# Individual Component Access
print("📚 Access individual components:")
print("   • rag = pipeline.rag_pipeline")
print("   • dwsim = pipeline.dwsim_pipeline") 
print("   • exporter = pipeline.exporter")
print()

# Custom Simulations
print("🧪 Run custom simulations:")
custom_case = {
    "name": "custom_test",
    "type": "reactor", 
    "components": ["hydrogen", "nitrogen"],
    "description": "Custom ammonia synthesis reactor"
}

print(f"Running custom simulation: {custom_case['name']}")
custom_result = pipeline.dwsim_pipeline.run_single_simulation(custom_case)
if custom_result:
    print(f"   ✅ Custom simulation completed: {custom_result['success']}")

print()
print("📁 All results automatically saved as CSV files in ./results/ directory")
print("🔄 Run any cell again to re-execute that specific pipeline component")

In [ ]:
# Clean up and reset (optional)
# pipeline.clean_all_results()  # Uncomment to clean all previous results

print(" PyNucleus Pipeline is ready!")
print(" Use the functions above to run different parts of the pipeline")
print(" All results are automatically exported to CSV files")
print(" You can run any cell multiple times to re-execute components")

In [ ]:
# Complete pipeline
results = pipeline.run_complete_pipeline()

# Individual components
rag_results = pipeline.run_rag_only()
dwsim_results = pipeline.run_dwsim_only()

# Utilities
pipeline.quick_test()
pipeline.view_results_summary()
pipeline.print_pipeline_status()

This is the last step in the code for version control purpose

In [49]:
from datetime import datetime

# Log end time
with open("update_log.txt", "a") as f:
    f.write(f"\n {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} changes made and pushed to origin main\n")

# Simple GitHub update function
def update_github(branch_name="main"):
    print(f"🔄 Starting GitHub update for branch: {branch_name}...")
    !git add .
    print("📦 Files added to staging")
    !git commit -m "Update: $(date +'%Y-%m-%d %H:%M:%S')"
    print("💾 Changes committed")
    !git push origin {branch_name}
    print(f"✅ Changes pushed to GitHub branch '{branch_name}' successfully!")

# To use it, just run with your desired branch name:
update_github("main-rag")# Or use default main branch:
# update_github()

🔄 Starting GitHub update for branch: Main-+-RAG...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


📦 Files added to staging
[main 556c75b] Update: 2025-06-10 16:18:36


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 43 files changed, 5396 insertions(+), 763 deletions(-)
 delete mode 100644 chunk_reports/faiss_analysis_20250610_011345.txt
 delete mode 100644 chunk_reports/faiss_analysis_20250610_011713.txt
 delete mode 100644 chunk_reports/faiss_analysis_20250610_011731.txt
 delete mode 100644 chunk_reports/faiss_analysis_20250610_012518.txt
 create mode 100644 core_modules/pipeline/__init__.py
 create mode 100644 core_modules/pipeline/pipeline_dwsim.py
 create mode 100644 core_modules/pipeline/pipeline_export.py
 create mode 100644 core_modules/pipeline/pipeline_rag.py
 create mode 100644 core_modules/pipeline/pipeline_utils.py
 delete mode 100644 docker_config/Dockerfile
 delete mode 100644 docker_config/docker-compose.yml
 create mode 100644 dwsim_rag_integration/README.md
 create mode 100644 dwsim_rag_integration/__init__.py
 create mode 100644 dwsim_rag_integration/config/Dockerfile.dwsim-service
 create mode 100644 dwsim_rag_integration/config/__init__.py
 create mode 100644 dwsim_rag_integr

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✅ Changes pushed to GitHub branch 'Main-+-RAG' successfully!


from datetime import datetime

# Log end time
with open("update_log.txt", "a") as f:
    f.write(f"\n {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} changes made and pushed to origin main\n")

# Simple GitHub update function
def update_github():
    !git add .
    !git commit -m "Update: Adding all files to repository"
    !git push origin main
    print("All files pushed to GitHub successfully!")

# To use it, just run:
update_github()